# Twitter Lists - Filtering

### Global Packages

In [39]:
import pandas as pd

# Local imports
from filter_lists import ListReader, ListFilter


#### Parameters and Constants

In [54]:
PATH = "C:/Users/fdmol/Desktop/Energy-Lab/twitter_search/twitter_search/data/raw_data"
COLS_TO_KEEP = ["user_id", "list_id", "name", "description"]

LISTS_KEYWORDS = ["air", "pollution", "earth", "climate", "smog"]

#### Functions and Classes

#### Pipeline

In [82]:
print("Reading JSON to create Lists dataframe...")
print(f"Reading {PATH}")
list_reader = ListReader()
lists_df = list_reader.create_df()

print("Filtering dataframe for relevant lists")
list_filter = ListFilter(lists_df)
lists_df["relevant"] = lists_df.apply(list_filter.is_relevant, axis=1)
relevant_lists = lists_df.loc[lists_df["relevant"].isin([True]), :]
print("Done!")


In [84]:
relevant_lists

,user_id,list_id,name,description,relevant
1,1410225032354172932,1732746835570430409,No to Air pollution,Stop Air pollution Stop Toxic Gases,True
10,1410225032354172932,1518253847445217280,Pollution,"Accounts/Posts related to land, air & water po...",True
14,1410225032354172932,194782213,EARTH 🌍 PROTECTING GROUP,TO PROTECT EARTH 🌍._ JAI SRI RAM.,True
113,1247376074272641025,1765717997334024210,Air quality,,True
114,1247376074272641025,1743504404563853614,Air pollution Researcher,People who are actively working to solve the a...,True
188,4429482316,766501326650048512,Smokey's friends,List for interesting people in the air quality...,True
192,4429482316,745346183120420865,Air Quality Experts,,True
193,4429482316,710779808536367104,smog 2016,,True
196,4429482316,718131431600484352,Air People,,True
